# GDP ETL Practice!

## Step 1 !
1. https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29 에 있는 표에서 국가별 GDP를 긁어오자 !
2. 긁어온 GDP를 확인할 수 있는 테이블(데이터프레임?)을 만들자 !
3. 해당 테이블에는 GDP가 높은 순으로 표시되어야 한다 !

In [1]:
# import field
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# 페이지에 요청을 떄려보자
response = requests.get(
    'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
print(f'response: {response}')
print(f'response.content: {response.content}')

response: <Response [200]>
response.content: b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of countries by GDP (nominal) - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinne

In [3]:
# html파서로 파싱 !
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of countries by GDP (nominal) - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-dis

In [4]:
# GDP 테이블을 찾아보자 !
# 해당 페이지에는 테이블이 하나 밖에 없음
# 클래스명이 wikitable인 태그
gdp_table = soup.find('table', {'class': 'wikitable'})
# 각 행을 파싱 <- <tr> 태그
rows = gdp_table.find_all('tr')

# 몇 번째 행부터 정보를 가져올까 ?
for i in range(10):
    print(f'rows[{i}]: {rows[i]}')

# row[3]부터 미국 시작 !

rows[0]: <tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th rowspan="2">Country/Territory
</th>
<th colspan="2"><a href="/wiki/International_Monetary_Fund" title="International Monetary Fund">IMF</a><sup class="reference" id="cite_ref-GDP_IMF_2-2"><a href="#cite_note-GDP_IMF-2">[1]</a></sup><sup class="reference" id="cite_ref-15"><a href="#cite_note-15">[13]</a></sup>
</th>
<th colspan="2"><a href="/wiki/World_Bank" title="World Bank">World Bank</a><sup class="reference" id="cite_ref-16"><a href="#cite_note-16">[14]</a></sup>
</th>
<th colspan="2"><a href="/wiki/United_Nations" title="United Nations">United Nations</a><sup class="reference" id="cite_ref-UN_17-0"><a href="#cite_note-UN-17">[15]</a></sup>
</th></tr>
rows[1]: <tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th>Forecast</th>
<th>Year</th>
<th>Estimate</th>
<th>Year</th>
<th>Estimate</th>
<th>Year
</th></tr>
rows[2]: <tr class="static-row-header" style="font-w

In [5]:
# 값이 맞는지 한번 찍어보자
for row in rows[3:8]:
    eles = row.find_all('td')
    country = eles[0].text.strip()  # 혹시 모를 공백이..
    gdp_forecast = eles[1].text.strip()
    print(country)
    print(gdp_forecast)
    # 콤마 제거 필요, int
    gdp_forecast = int(gdp_forecast.replace(',', ''))
    print(gdp_forecast)

# 완료 ! => 이제 모든 나라에 대해서 실행

United States
28,781,083
28781083
China
18,532,633
18532633
Germany
4,591,100
4591100
Japan
4,110,452
4110452
India
3,937,011
3937011


In [6]:
# 나라, gdp df를 만들자 (혹시 모르니 연도도 추가하자)
# 연도 텍스트에 이상한게 붙어있는게 있다 !? [n 11]2024 이런거
# 뒤에서부터 4글자만 긁자 !
countries = []
gdps = []
years = []

for row in rows[3:]:
    eles = row.find_all('td')  # td태그 달린 애들 중 [0][1]만 쓸 것임 !
    # 나라 추가
    country = eles[0].text.strip()
    countries.append(country)
    # gdp, 연도 추가
    # - 가 있음 ! => 적절한 처리(아래)
    if eles[1].text.strip().replace(',', '') != '—':
        gdp_forecast = int(eles[1].text.strip().replace(',', ''))
        gdps.append(gdp_forecast)
        year = int(eles[2].text.strip()[-4:])
        years.append(year)
    else:  # 측정이 안됐을 경우
        gdps.append(np.nan)
        years.append(np.nan)

print(f'len(countries), len(gdps), len(years): {
      len(countries), len(gdps), len(years)}')

# 데이터프레임 생성
gdp_df = pd.DataFrame(
    {
        'Country': countries,
        'GDP': gdps,
        'Year': years
    }
)

# 확인 !
gdp_df

len(countries), len(gdps), len(years): (213, 213, 213)


,Country,GDP,Year
0,United States,28781083.0,2024.0
1,China,18532633.0,2024.0
2,Germany,4591100.0,2024.0
3,Japan,4110452.0,2024.0
4,India,3937011.0,2024.0
...,...,...,...
208,Marshall Islands,305.0,2024.0
209,Cook Islands,NaN,NaN
210,Nauru,161.0,2024.0
211,Montserrat,NaN,NaN


In [7]:
# Region 별로 TOP 5 => Region 컬럼도 필요할듯 !
# GDP의 단위는 1B USD이어야 하고 소수점 2자리까지만 표시 => 단위 변환이 필요

# Region 테이블이 필요할듯 ?
# 위키를 참고하자 !
# https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_the_United_Nations_geoscheme

response = requests.get(
    'https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_the_United_Nations_geoscheme')
print(f'response: {response}')
print(f'response.content: {response.content}')

soup = BeautifulSoup(response.content, 'html.parser')

region_table = soup.find('table', {'class': 'wikitable'})
# parsing
rows = region_table.find_all('tr')
print(rows[1])
countries = []
geograpical_subregions = []
intermediary_regions = []
continental_regions = []


for row in rows[1:]:
    eles = row.find_all('td')
    # 나라
    country = eles[0].text.strip()
    countries.append(eles[0].text.strip())
    # geographical subregion
    geograpical_subregion = eles[1].text.strip()
    if geograpical_subregion == '—':
        geograpical_subregion = np.nan
    geograpical_subregions.append(geograpical_subregion)
    # intermediary region
    intermediary_region = eles[2].text.strip()
    if intermediary_region == '—':
        intermediary_region = np.nan
    intermediary_regions.append(intermediary_region)
    # continental region
    continental_region = eles[3].text.strip()
    if continental_region == '—':
        continental_region = np.nan
    continental_regions.append(continental_region)
    print(f'----- now country: {country}')
    print(f'geograpical_subregion, intermediary_region, continental_region')
    print(f'{geograpical_subregion, intermediary_region, continental_region}')

print(f'end={len(geograpical_subregions), len(
    intermediary_regions), len(continental_regions)}')
# df 만들기
region_df = pd.DataFrame(
    {
        'Country': countries,
        'Geograpical Subregion': geograpical_subregions,
        'Intermediary Region': intermediary_regions,
        'Continental Region': continental_regions
    }
)

response: <Response [200]>
response.content: b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of countries and territories by the United Nations geoscheme - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-di

In [8]:
region_df

,Country,Geograpical Subregion,Intermediary Region,Continental Region
0,Algeria,Northern Africa,NaN,Africa
1,Egypt,Northern Africa,NaN,Africa
2,Libya,Northern Africa,NaN,Africa
3,Morocco,Northern Africa,NaN,Africa
4,Sudan,Northern Africa,NaN,Africa
...,...,...,...,...
243,Samoa,Polynesia,NaN,Oceania
244,Tokelau,Polynesia,NaN,Oceania
245,Tonga,Polynesia,NaN,Oceania
246,Tuvalu,Polynesia,NaN,Oceania


In [9]:
# 1. Region 별로 TOP 5
# 2. gdp 단위 변환 필요

# gdp_df, region_df를 합치자 !
# LEFT OUTER JOIN !

joined_df = pd.merge(gdp_df, region_df, on='Country', how='left')

joined_df

,Country,GDP,Year,Geograpical Subregion,Intermediary Region,Continental Region
0,United States,28781083.0,2024.0,NaN,NaN,NaN
1,China,18532633.0,2024.0,Eastern Asia,NaN,Asia
2,Germany,4591100.0,2024.0,Western Europe,NaN,Europe
3,Japan,4110452.0,2024.0,Eastern Asia,NaN,Asia
4,India,3937011.0,2024.0,Southern Asia,NaN,Asia
...,...,...,...,...,...,...
208,Marshall Islands,305.0,2024.0,Micronesia,NaN,Oceania
209,Cook Islands,NaN,NaN,Polynesia,NaN,Oceania
210,Nauru,161.0,2024.0,Micronesia,NaN,Oceania
211,Montserrat,NaN,NaN,Caribbean,Latin America and the CaribbeanNorth America,Americas


In [10]:
# 매칭이 안되는 값이 조금 있는 것 같다 !
joined_df.isna().sum()
# Continental Region만 조금 달아주자...

Country                    0
GDP                       19
Year                      19
Geograpical Subregion     33
Intermediary Region      130
Continental Region        33
dtype: int64

In [48]:
region_df['Continental Region'].unique()

array(['Africa', 'Americas', 'Antarctica', 'Asia', 'Europe', 'Oceania'],
      dtype=object)

In [14]:
joined_df[joined_df['Continental Region'].isna()]['Country'].values

array(['United States', 'United Kingdom', 'France', 'Russia',
       'South Korea', 'Turkey', 'Taiwan', 'Vietnam', 'Iran', 'Hong Kong',
       'Czech Republic', 'Venezuela', 'Ivory Coast', 'Tanzania',
       'DR Congo', 'Myanmar', 'Macau', 'Bolivia', 'Palestine', 'Moldova',
       'Brunei', 'Congo', 'Laos', 'North Korea', 'Kosovo', 'Syria',
       'Eswatini', 'Cape Verde', 'Zanzibar', 'East Timor', 'Sint Maarten',
       'São Tomé and Príncipe', 'Micronesia'], dtype=object)

In [15]:
continents = ['Americas', 'Europe', 'Europe', 'Europe',
              'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia',
              'Europe', 'Americas', 'Africa', 'Africa',
              'Africa', 'Asia', 'Asia', 'Americas', 'Asia', 'Europe',
              'Asia', 'Africa', 'Asia', 'Asia', 'Europe', 'Asia',
              'Africa', 'Africa', 'Africa', 'Asia', 'Americas',
              'Africa', 'Oceania']

new_df = pd.DataFrame(
    {
        'Country': joined_df[joined_df['Continental Region'].isna()]['Country'].values,
        'Geograpical Subregion': [np.nan for _ in range(33)],
        'Intermediary Region': [np.nan for _ in range(33)],
        'Continental Region': continents
    }
)
new_df

,Country,Geograpical Subregion,Intermediary Region,Continental Region
0,United States,NaN,NaN,Americas
1,United Kingdom,NaN,NaN,Europe
2,France,NaN,NaN,Europe
3,Russia,NaN,NaN,Europe
4,South Korea,NaN,NaN,Asia
5,Turkey,NaN,NaN,Asia
6,Taiwan,NaN,NaN,Asia
7,Vietnam,NaN,NaN,Asia
8,Iran,NaN,NaN,Asia
9,Hong Kong,NaN,NaN,Asia


In [16]:
region_df = pd.concat([region_df, new_df], ignore_index=True)
region_df

,Country,Geograpical Subregion,Intermediary Region,Continental Region
0,Algeria,Northern Africa,NaN,Africa
1,Egypt,Northern Africa,NaN,Africa
2,Libya,Northern Africa,NaN,Africa
3,Morocco,Northern Africa,NaN,Africa
4,Sudan,Northern Africa,NaN,Africa
...,...,...,...,...
276,Zanzibar,NaN,NaN,Africa
277,East Timor,NaN,NaN,Asia
278,Sint Maarten,NaN,NaN,Americas
279,São Tomé and Príncipe,NaN,NaN,Africa


In [17]:
joined_df = pd.merge(gdp_df, region_df, on='Country', how='left')
joined_df

,Country,GDP,Year,Geograpical Subregion,Intermediary Region,Continental Region
0,United States,28781083.0,2024.0,NaN,NaN,Americas
1,China,18532633.0,2024.0,Eastern Asia,NaN,Asia
2,Germany,4591100.0,2024.0,Western Europe,NaN,Europe
3,Japan,4110452.0,2024.0,Eastern Asia,NaN,Asia
4,India,3937011.0,2024.0,Southern Asia,NaN,Asia
...,...,...,...,...,...,...
208,Marshall Islands,305.0,2024.0,Micronesia,NaN,Oceania
209,Cook Islands,NaN,NaN,Polynesia,NaN,Oceania
210,Nauru,161.0,2024.0,Micronesia,NaN,Oceania
211,Montserrat,NaN,NaN,Caribbean,Latin America and the CaribbeanNorth America,Americas


In [18]:
joined_df.isna().sum()

Country                    0
GDP                       19
Year                      19
Geograpical Subregion     33
Intermediary Region      130
Continental Region         0
dtype: int64

In [19]:
# gdp 단위 변환
joined_df['GDP'] = (joined_df['GDP']/1000).round(2)
joined_df

,Country,GDP,Year,Geograpical Subregion,Intermediary Region,Continental Region
0,United States,28781.08,2024.0,NaN,NaN,Americas
1,China,18532.63,2024.0,Eastern Asia,NaN,Asia
2,Germany,4591.10,2024.0,Western Europe,NaN,Europe
3,Japan,4110.45,2024.0,Eastern Asia,NaN,Asia
4,India,3937.01,2024.0,Southern Asia,NaN,Asia
...,...,...,...,...,...,...
208,Marshall Islands,0.30,2024.0,Micronesia,NaN,Oceania
209,Cook Islands,NaN,NaN,Polynesia,NaN,Oceania
210,Nauru,0.16,2024.0,Micronesia,NaN,Oceania
211,Montserrat,NaN,NaN,Caribbean,Latin America and the CaribbeanNorth America,Americas
